In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import sys
sys.path.insert(0, '../../')
import seaborn as sns
import numpy as np

from JKBio.Helper import *
from JKBio.helper import pyDESeq2

from bokeh.plotting import *
from bokeh.models import HoverTool
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
#from umap import UMAP

output_notebook()
%load_ext autoreload
%matplotlib inline
%autoreload 2
%load_ext rpy2.ipython

In [ ]:
project = "Muhar_Slamseq"
version = "v2"

In [ ]:
## downloading the data (need to do it for more samples than what is displayed here)

In [ ]:
! mkdir ../../data/$project/MYCpaper && cd ../../data/$project/MYCpaper && fastq_dump SRR5806781 && fastq_dump SRR5806783 && fastq_dump SRR5806785 && fastq_dump SRR5806780 && fastq_dump SRR5806782 && fastq_dump SRR5806784  && cd -

In [ ]:
folder_bams = "../../data/"+project+"/MYCpaper/"
bams = ! ls $folder_bams
bams

In [ ]:
for val in bams[1:]:
    print(val)
    os.system('../../TrimGalore-0.6.5/trim_galore -stringency 3 \
    --illumina '+folder_bams+val + ' -o '+folder_bams) 

In [ ]:
ls -al ../../data/$project/res/count

In [ ]:
for val in bams[1:]:
    print(val)
    os.system('slamdunk all -r ../../data/ref/ref_genome.fa -b ../../data/'+project+'/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/'+project+'/res/ -t 8 -c 2 '+folder_bams+val.split('.')[0]+'_trimmed.fq')

In [ ]:
ls -al ../../data/$project/res/filter

In [ ]:
for val in bams[1:]:
    print(val)
    os.system('slamdunk count -r ../../data/ref/ref_genome.fa \
    -b ../../data/'+project+'/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/'+project+'/res/count/ -t 8 -c 2 -s ../../data/'+project+'/res/snp \
    ../../data/'+project+'/res/filter/'+ val.split('.')[0]+"_trimmed_slamdunk_mapped_filtered.bam")

# we first need to recreate out RNA matrix from slamdunk's tcount files

In [ ]:
ls ../data/$project

In [ ]:
files = ! ls $location/MOLM13 $location/MYC-K562-HCT116 $location/K562
files

extracting K562 only

In [ ]:
data={}
for file in files:
    data[file.split('/')[-1].split('.')[0]]= pd.read_csv(file, sep='\t', comment='#',header=0)

In [ ]:
files

In [ ]:
prev = -2
for k,val in data.items():
    if len(set(val.Name)) != prev and prev!=-2: 
        raise ValueError('we do not have the same number of genes in each file')
    prev = len(set(val.Name))

![](images/conversionrates.png)

In [ ]:
data.keys()

In [ ]:
val

In [ ]:
flag_var= 100

readcounts = {i:[0]*len(data) for i in val.Name.unique()}
tccounts = {i:[0]*len(data) for i in val.Name.unique()}

In [ ]:
for n, (_,val) in enumerate(data.items()):
    val = val.sort_values(by="Name")
    j  = 0
    #print('              ',end='\r')
    readcount = [val.iloc[0].ReadCount]
    tccount = [val.iloc[0].TcReadCount]
    prevname = val.iloc[0].Name
    for _, v in val.iloc[1:].iterrows():
        if v.Name==4609:
            print(v.ReadCount,v.TcReadCount)
            print(readcount,tccount)
        if v.Name == prevname:
            readcount.append(v.ReadCount)
            tccount.append(v.TcReadCount)
        else:
            readcounts[prevname][n] = np.sum(readcount)
            tccounts[prevname][n] = np.sum(tccount)
            #if np.var(readcount) > flag_var:
            #    print("pb with "+str(v.Name))
            prevname=v.Name
            j+=1
            #print(j,end='\r')
            readcount = [v.ReadCount]
            tccount = [v.TcReadCount]
            
files = [*data]

In [ ]:
val.Name

In [ ]:
files

In [ ]:
readcounts[4609]

In [ ]:
names = {
"GSM2691910": "K562_MYC-AID_DMSO_rep-1",
"GSM2691911": "K562_MYC-AID_DMSO_rep-2",
"GSM2691912": "K562_MYC-AID_DMSO_rep-3",
"GSM2691913": "K562_MYC-AID_IAA_rep-1",
"GSM2691914": "K562_MYC-AID_IAA_rep-2",
"GSM2691915": "K562_MYC-AID_IAA_rep-3",
"GSM2691916": "HCT116_MYC-AID_DMSO_rep-1",
"GSM2691917": "HCT116_MYC-AID_DMSO_rep-2",
"GSM2691918": "HCT116_MYC-AID_DMSO_rep-3",
"GSM2691919": "HCT116_MYC-AID_IAA_rep-1",
"GSM2691920": "HCT116_MYC-AID_IAA_rep-2",
"GSM2691921": "HCT116_MYC-AID_IAA_rep-3",
"GSM2691882": "MOLM-13_DMSO_rep-1_exp-2",
"GSM2691883": "MOLM-13_DMSO_rep-2_exp-2",
"GSM2691884": "MOLM-13_DMSO_rep-3_exp-2",
"GSM2691885": "MOLM-13_JQ1_rep-1_exp-2",
"GSM2691886": "MOLM-13_JQ1_rep-2_exp-2",
"GSM2691887": "MOLM-13_JQ1_rep-3_exp-2",
"GSM2691888": "MOLM-13_6nM_NVP-2_rep-1_exp-2",
"GSM2691889": "MOLM-13_6nM_NVP-2_rep-2_exp-2",
"GSM2691890": "MOLM-13_6nM_NVP-2_rep-3_exp-2",
"GSM2691891": "MOLM-13_60nM_NVP-2_rep-1_exp-2",
"GSM2691892": "MOLM-13_60nM_NVP-2_rep-2_exp-2",
"GSM2691893": "MOLM-13_JQ1+6nM_NVP-2_rep-1_exp-2",
"GSM2691894": "MOLM-13_JQ1+6nM_NVP-2_rep-2_exp-2",
"GSM2691895": "MOLM-13_JQ1+6nM_NVP-2_rep-3_exp-2"

In [ ]:
tccounts = tccounts.rename(columns=names)
readcounts = readcounts.rename(columns=names)
readcounts

In [ ]:
readcounts = pd.DataFrame(data=readcounts,columns=val.Name.unique(),index=data.keys()).T
tccounts = pd.DataFrame(data=tccounts,columns=val.Name.unique(),index=data.keys()).T

In [ ]:
readcounts.loc[4609]

In [ ]:
names, _ = h.convertGenes(readcounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
readcounts.index = names

In [ ]:
names, _ = h.convertGenes(tccounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
tccounts.index = names

In [ ]:
alltccounts = tccounts
allreadcounts = readcounts

## subset only

In [ ]:
subsetname = "K562"
subset = []
tccounts = alltccounts.columns[subset]
readcounts = allreadcounts.columns[subset]

In [ ]:
minvar_toremove = 2
mincount_toremove = 10

In [ ]:
nottodrop=np.argwhere(tccounts.values.var(1)>minvar_toremove).ravel()
tccounts = tccounts.iloc[nottodrop]
readcounts = readcounts.iloc[nottodrop]

In [ ]:
nottodrop=np.argwhere(readcounts.values[:,:-1].max(1)>=mincount_toremove).ravel()
tccounts = tccounts.iloc[nottodrop]
readcounts = readcounts.iloc[nottodrop]

In [ ]:
tccounts

In [ ]:
tccounts.to_csv("../results/"+project+"/"+version+"_"+subsetname+"_"+minvar_toremove+"_"+
mincount_toremove+"_tccounts.csv")
readcounts.to_csv("../results/"+project+"/"+version+"_"+subsetname+"_"+minvar_toremove+"_"+
mincount_toremove+"_readcounts.csv")

In [ ]:
mtccounts = pd.DataFrame()
mreadcounts = pd.DataFrame()
for i in range(4):
    mtccounts[tccounts.columns[i*3].split("_rep")[0]] = tccounts[tccounts.columns[(i*3):(i*3)+3]].mean(1)
    mreadcounts[tccounts.columns[i*3].split("_rep")[0]] = readcounts[readcounts.columns[(i*3):(i*3)+3]].mean(1)

In [ ]:
mtccounts.to_csv("../results/"+project+"/"+version+"_"+subsetname+"_"+minvar_toremove+"_"+
mincount_toremove+"_mean_tccounts.csv")
mreadcounts.to_csv("../results/"+project+"/"+version+"_"+subsetname+"_"+minvar_toremove+"_"+
mincount_toremove+"_mean_readcounts.csv")

In [ ]:
mtccounts = pd.DataFrame()
mreadcounts = pd.DataFrame()
for i in range(4):
    mtccounts[tccounts.columns[i*3].split("_rep")[0]] = tccounts[tccounts.columns[(i*3):(i*3)+3]].median(1)
    mreadcounts[tccounts.columns[i*3].split("_rep")[0]] = readcounts[readcounts.columns[(i*3):(i*3)+3]].median(1)

In [ ]:
mtccounts.to_csv("../results/"+project+"/"+version+"_"+subsetname+"_"+minvar_toremove+"_"+
mincount_toremove+"_median_tccounts.csv")
mreadcounts.to_csv("../results/"+project+"/"+version+"_"+subsetname+"_"+minvar_toremove+"_"+
mincount_toremove+"_median_readcounts.csv")

## SLAM-seq
All SLAM-seq assays were performed at 60-70% confluency for adherent cells or
60% of the maximum cell density counted on a hemocytometer for suspension cells. 5-7h
prior to each assay, growth medium was aspirated and replaced. Unless stated otherwise,
cells were pre-treated with indicated small molecule inhibitors or 100µM IAA for 30 min
to pre-establish full target inhibition or degradation. Newly synthesized RNA was labeled
for indicated time spans (45 min or 60 min) at a final concentration of 100µM 4-
thiouridine (4sU, Carbosynth). Adherent cells were harvested by direct snap-freezing of
plates on dry ice. Suspension cells were spun down and immediately snap-frozen. RNA
extraction was performed using the RNeasy Plus Mini Kit (Qiagen). 

Total RNA was subjected to alkylation by iodoacetamide (Sigma, 10mM) for 15 min and RNA was repurified by ethanol precipitation. 500ng alkylated RNA were used as input for generating
3’-end mRNA sequencing libraries using a commercially available kit (QuantSeq 3′
mRNA-Seq Library Prep Kit FWD for Illumina and PCR Add-on Kit for Illumina,
Lexogen). Deep sequencing was performed using HiSeq1500 and HiSeq2500 platforms
(Illumina).

In [ ]:
tccounts

# lets look at the similarity accross replicates

we have some outliers, else it seems to make some sense and on average, to be extremelly similar!

In [ ]:
tccounts.columns

In [ ]:
%matplotlib inline
sns.heatmap(tccounts.corr(), 
            xticklabels=tccounts.columns.values,
            yticklabels=tccounts.columns.values)

In [ ]:
scatter(UMAP(n_neighbors=2,min_dist=0.1).fit_transform(tccounts.values.T),labels=tccounts.columns, radi=1)

In [ ]:
%matplotlib inline
sns.heatmap(readcounts.corr(), 
            xticklabels=readcounts.columns.values,
            yticklabels=readcounts.columns.values)

read the paper for MV411 QC steps.

### A quick detour to look for clustering

## Getting the Core TF information

In [ ]:
ctf=pd.read_csv('../data/CTF.csv',header=None)[0].values.tolist()
ctf

In [ ]:
set(ctf) & set(tccounts.index)

# Comparisons between two conditions 

how core transcription factors change accross time when the cell is with JQ1 conditions

we are looking at the difference between production of each CTF compared to the average RNA production with JQ1 and without JQ1

## DESEQ ANALYSIS

In [ ]:
readcounts["genes"] = readcounts.index
readcounts = readcounts.reset_index(drop=True)
tccounts["genes"] = tccounts.index
tccounts = tccounts.reset_index(drop=True)

In [ ]:
sideA = [[1,1,1,0,0,0,0,0,0,0,0,0],
         [0,0,0,1,1,1,0,0,0,0,0,0]]
a = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','JQ1'], 
                      data=np.array(sideA,dtype=bool).T)

In [ ]:
sideB = [[0,0,0,0,0,0,1,1,1,0,0,0],
         [0,0,0,0,0,0,0,0,0,1,1,1]]
b = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','JQ1'], 
                      data=np.array(sideB,dtype=bool).T)

### Differential gene expression analysis, PCA and GO-term enrichment

For gene-level analysis, raw reads mapped to different UTR annotations of the same gene were summed up by Entrez Gene ID. Pilot studies of K562 cells with kinase inhibitors were performed as single experiments. 

Analysis of differential gene expression was restricted to genes with ≥ 10 reads in at least one condition for 50bp sequencing runs (flavopiridol and DMSO) or ≥ 20 reads in at least one condition for 100bp sequencing runs (mk2206, trametinib, nilotinib, trametinib + mk2206 and DMSO). For estimating differential expression, a pseudo-count of 1 raw read was added to all genes.

#### All other SLAM-seq experiments were performed in triplicates and analyzed as follows. 
Differential gene expression calling was performed on raw read counts with ≥ 2 T>C conversions using DESeq2 (version 1.14.1) with default settings, and with size factors estimated on corresponding total mRNA reads for global normalization.

Downstream analysis was restricted to genes passing all internal filters for FDR estimation by DESeq2. Principal component analysis was performed after variance
stabilizing transformation on the 500 most variable genes across all conditions of a given
experiment. GO-term enrichment analysis was performed on genes significantly and strongly downregulated (FDR ≤ 0.1, log2FC ≤ -1) in SLAM-seq upon IAA-treatment in K562MYC-AID + Tir1 by the PANTHER Overrepresentation Test (Fisher's Exact with FDR multiple test correction, release 20171205, http://pantherdb.org) on GO Ontology database Released 2017-12-27.

In [ ]:
tccounts.columns = [i.replace('-','.') for i in tccounts.columns]

In [ ]:
a = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array(sideA[0]+[1],np.bool)|np.array(sideA[1]+[1],np.bool)]],
                         design_matrix=a[np.array(sideA[0],np.bool)| np.array(sideA[1],np.bool)],
                         design_formula="~DMSO - JQ1",
                         gene_column="genes")

In [ ]:
b = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array(sideB[0]+[1],np.bool)|np.array(sideB[1]+[1],np.bool)]],
                         design_matrix=b[np.array(np.array(sideB[0],np.bool)| np.array(sideB[1],np.bool)],
                         design_formula="~DMSO - JQ1",
                         gene_column="genes")

In [ ]:
a.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array(sideA[1]+[0],np.bool)]].values+1), 1)))

In [ ]:
b.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array(sideB[1]+[0],np.bool)]].values+1), 1)))

In [ ]:
a.run_deseq()
a.get_deseq_result()
a = a.deseq_result
a.pvalue = np.nan_to_num(np.array(a.pvalue), 1)
a.log2FoldChange = np.nan_to_num(np.array(a.log2FoldChange), 0)
a.log2FoldChange = -a.log2FoldChange
a["gene_id"] = a.genes

In [ ]:
b.run_deseq()
b.get_deseq_result()
b = b.deseq_result
b.pvalue = np.nan_to_num(np.array(b.pvalue), 1)
b.log2FoldChange = np.nan_to_num(np.array(b.log2FoldChange), 0)
b.log2FoldChange = -b.log2FoldChange
b["gene_id"] = b.genes

# PLOTS

## Scatter plot comparison

In [ ]:
mix = pd.DataFrame()

In [ ]:
mix["gene_id"] = a["gene_id"]

In [ ]:
mix['x'] = - a.log2FoldChange
mix['y'] = b.log2FoldChange

In [ ]:
h.scatter(mix[['x','y']].values[:12000], 
               mix['gene_id'].values.tolist()[:12000], radi= 0.06, alpha=0.3,
              colors = [0 if i in ctf else 1 for i in mix['gene_id'].values.tolist()[:12000]],
         folder = )

In [ ]:
a.toc

we can conclude that we get similar results to the slamseq myc paper although it seems that our values are a bit skewed toward higher expression than what is on the slamseq paper. It mightt be explained by the pseudo count of 1 that I did not set. Because I think it would highly bias the DESeq algorithm.

## Whiskers plot for each. 
comparing ctf change vs other genes 

In [ ]:
res = a[a.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("MV411")

In [ ]:
sns.boxplot(data=res,x='type',y='baseMean').set_title("MV411")

In [ ]:
res = K562res[K562res.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("K562")

In [ ]:
sns.boxplot(data=res,x='type',y='baseMean').set_title("K562")

In [ ]:
show(h.volcano(MV411res,tohighlight=ctf, searchbox=True, title="MV411"))
show(h.volcano(K562res,tohighlight=ctf, searchbox=True, title="K562"))

In [ ]:
! mkdir '../results/MV411_K562-JQ1' 

In [ ]:
MV411res.to_csv("../results/MV411_K562-JQ1/MV411.csv")
K562res.to_csv("../results/MV411_K562-JQ1/K562.csv")

In [ ]:
readcounts.to_csv('../results/MV411_K562-JQ1/readcounts.csv')
tccounts.to_csv('../results/MV411_K562-JQ1/tccounts.csv')

In [ ]:
cols = list(set([i.split('_rep')[0] for i in tccounts.columns]))
cols.remove('genes')
cols

In [ ]:
meantc = pd.DataFrame()
for val in cols:
    meantc[val] = tccounts[[i for i in tccounts.columns if val in i]].mean(1)
meanread = pd.DataFrame()
for val in cols:
    meanread[val] = readcounts[[i for i in readcounts.columns if val in i]].mean(1)
medread = pd.DataFrame()
for val in cols:
    medread[val] = readcounts[[i for i in readcounts.columns if val in i]].median(1)
medtc = pd.DataFrame()
for val in cols:
    medtc[val] = tccounts[[i for i in tccounts.columns if val in i]].median(1)

In [ ]:
medtc.to_csv('../results/MV411_K562-JQ1/medtc.csv')
meantc.to_csv('../results/MV411_K562-JQ1/meantc.csv')
medread.to_csv('../results/MV411_K562-JQ1/medread.csv')
meanread.to_csv('../results/MV411_K562-JQ1/meanread.csv')

## WITH MYC SAMPLES

In [ ]:
design_K562_MYC = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,1,1,1,0,0,0,0,0,0]],dtype=bool).T)

In [ ]:
design_HCT116_MYC = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[0,0,0,0,0,0,1,1,1,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,1,1,1]],dtype=bool).T)

In [ ]:
scaling6 = np.log2((tccounts.mean()/readcounts.mean())[[2,3]].mean() / (tccounts.mean()/readcounts.mean())[7:11].mean())
scaling6

In [ ]:
deseq_K562.run_estimate_size_factors(normMatrix = (readcounts[readcounts.columns[
        np.array([1,1,1,1,1,1,0,0,0,0,0,0,0],np.bool)]].values+1))
deseq_HCT116.run_estimate_size_factors(normMatrix = (readcounts[readcounts.columns[
        np.array([0,0,0,0,0,0,1,1,1,1,1,1,0],np.bool)]].values+1))

In [ ]:
deseq_K562.run_deseq()
deseq_HCT116.run_deseq()

In [ ]:
deseq_K562.get_deseq_result()
deseq_HCT116.get_deseq_result()

In [ ]:
res_K562 = deseq_K562.deseq_result
res_HCT116 = deseq_HCT116.deseq_result

In [ ]:
res_K562.pvalue = np.nan_to_num(np.array(res_K562.pvalue), 1)
res_K562.log2FoldChange = np.nan_to_num(np.array(res_K562.log2FoldChange), 0)
res_HCT116.pvalue = np.nan_to_num(np.array(res_HCT116.pvalue), 1)
res_HCT116.log2FoldChange = np.nan_to_num(np.array(res_HCT116.log2FoldChange), 0)

In [ ]:
res_K562.log2FoldChange = -res_K562.log2FoldChange
res_HCT116.log2FoldChange = -res_HCT116.log2FoldChange

In [ ]:
res_K562["gene_id"] = res_K562.genes
res_HCT116["gene_id"] = res_HCT116.genes

In [ ]:
res6.log2FoldChange = res6.log2FoldChange - scaling6

In [ ]:
res_K562.to_csv('../data/MYC slamseq paper/K562_diffexp.csv')

In [ ]:
show(volcano(res_K562,tohighlight=ctf, searchbox=True, maxvalue=20,minpval=3, minlogfold=0.4))

In [ ]:
res_HCT116.to_csv('../data/MYC slamseq paper/HCT116_diffexp.csv')

In [ ]:
show(volcano(res_HCT116,tohighlight=ctf, searchbox=True, minpval=4, minlogfold=0.3))

## MOLM13

In [ ]:
sorting = [0,1,2,3,4,5,6,8,10,7,9,11,12,13,14]
tccounts = tccounts[tccounts.columns[sorting]]

In [ ]:
%matplotlib inline
sns.heatmap(tccounts.corr(), 
            xticklabels=tccounts.columns.values,
            yticklabels=tccounts.columns.values)

In [ ]:
readcounts = readcounts[readcounts.columns[sorting]]

In [ ]:
%matplotlib inline
sns.heatmap(readcounts.corr(), 
            xticklabels=readcounts.columns.values,
            yticklabels=readcounts.columns.values)

In [ ]:
tccounts.columns = [i.replace('-','.') for i in tccounts.columns]

In [ ]:
design1 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','JQ1'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,1,1,1,0,0,0,0,0,0,0,0]],dtype=bool).T)
design2 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','NVP2low'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,1,1,1,0,0,0,0,0]],dtype=bool).T)
design3 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','NVP2high'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,1,1,0,0,0]],dtype=bool).T)
design4 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','NVP2lowJQ1'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,0,0,1,1,1]],dtype=bool).T)

In [ ]:
deseq1 = pyDESeq2.pyDESeq2(count_matrix = tccounts[tccounts.columns[np.array([1,1,1,1,1,1,0,0,0,0,0,0,0,0,1],np.bool)]],
                         design_matrix=design1[np.array([1,1,1,1,1,1,0,0,0,0,0,0,0,0],np.bool)],
                         design_formula="~DMSO - JQ1",
                         gene_column="genes")
deseq2 = pyDESeq2.pyDESeq2(count_matrix = tccounts[tccounts.columns[np.array([1,1,1,0,0,0,1,1,1,0,0,0,0,0,1],np.bool)]],
                         design_matrix=design2[np.array([1,1,1,0,0,0,1,1,1,0,0,0,0,0],np.bool)],
                         design_formula="~DMSO - NVP2low",
                         gene_column="genes")
deseq3 = pyDESeq2.pyDESeq2(count_matrix = tccounts[tccounts.columns[np.array([1,1,1,0,0,0,0,0,0,1,1,0,0,0,1],np.bool)]],
                         design_matrix=design3[np.array([1,1,1,0,0,0,0,0,0,1,1,0,0,0],np.bool)],
                         design_formula="~DMSO - NVP2high",
                         gene_column="genes")
deseq4 = pyDESeq2.pyDESeq2(count_matrix = tccounts[tccounts.columns[np.array([1,1,1,0,0,0,0,0,0,0,0,1,1,1,1],np.bool)]],
                         design_matrix=design4[np.array([1,1,1,0,0,0,0,0,0,0,0,1,1,1],np.bool)],
                         design_formula="~DMSO - NVP2lowJQ1",
                         gene_column="genes")

In [ ]:
deseq1.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,1,1,1,0,0,0,0,0,0,0,0,0], np.bool)]].values+1), 1)))
deseq2.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,0,0,0,1,1,1,0,0,0,0,0,0], np.bool)]].values+1), 1)))
deseq3.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,0,0,0,0,0,0,1,1,0,0,0,0], np.bool)]].values+1), 1)))
deseq4.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,0,0,0,0,0,0,0,0,1,1,1,0], np.bool)]].values+1), 1)))

In [ ]:
deseq1.run_deseq()
deseq2.run_deseq()
deseq3.run_deseq()
deseq4.run_deseq()
deseq1.get_deseq_result()
deseq2.get_deseq_result()
deseq3.get_deseq_result()
deseq4.get_deseq_result()
res1 = deseq1.deseq_result
res2 = deseq2.deseq_result
res3 = deseq3.deseq_result
res4 = deseq4.deseq_result
res1.pvalue = np.nan_to_num(np.array(res1.pvalue), 1)
res1.log2FoldChange = np.nan_to_num(np.array(res1.log2FoldChange), 0)
res2.pvalue = np.nan_to_num(np.array(res2.pvalue), 1)
res2.log2FoldChange = np.nan_to_num(np.array(res2.log2FoldChange), 0)
res3.pvalue = np.nan_to_num(np.array(res3.pvalue), 1)
res3.log2FoldChange = np.nan_to_num(np.array(res3.log2FoldChange), 0)
res4.pvalue = np.nan_to_num(np.array(res4.pvalue), 1)
res4.log2FoldChange = np.nan_to_num(np.array(res4.log2FoldChange), 0)
res1.log2FoldChange = -res1.log2FoldChange
res2.log2FoldChange = -res2.log2FoldChange
res3.log2FoldChange = -res3.log2FoldChange
res4.log2FoldChange = -res4.log2FoldChange
res1["gene_id"] = res1.genes
res2["gene_id"] = res2.genes
res3["gene_id"] = res3.genes
res4["gene_id"] = res4.genes

### PLOTS

In [ ]:
res = res1[res1.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("JQ1")

In [ ]:
sns.boxplot(data=res,x='type',y='baseMean').set_title("JQ1")

In [ ]:
res = res2[res2.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("NVP2low")

In [ ]:
sns.boxplot(data=res,x='type',y='baseMean').set_title("NVP2low")

In [ ]:
res = res3[res3.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("NVP2high")

In [ ]:
sns.boxplot(data=res,x='type',y='baseMean').set_title("NVP2high")

In [ ]:
res = res4[res4.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("NVP2low+JQ1")

In [ ]:
sns.boxplot(data=res,x='type',y='baseMean').set_title("NVP2low+JQ1")

In [ ]:
show(h.volcano(res1,tohighlight=ctf, searchbox=True, title="JQ1"))
show(h.volcano(res2,tohighlight=ctf, searchbox=True, title="NVP2low"))
show(h.volcano(res3,tohighlight=ctf, searchbox=True, title="NVP2hgih"))
show(h.volcano(res4,tohighlight=ctf, searchbox=True, title="NVP2low+JQ1"))

we can conclude that we get similar results to the slamseq myc paper although it seems that our values are a bit skewed toward higher expression than what is on the slamseq paper. It mightt be explained by the pseudo count of 1 that I did not set. Because I think it would highly bias the DESeq algorithm.

In [ ]:
res1.to_csv("../results/MYCpaper_vMOLM13/jq1.csv")
res2.to_csv("../results/MYCpaper_vMOLM13/nvp2low.csv")
res3.to_csv("../results/MYCpaper_vMOLM13/nvp2high.csv")
res4.to_csv("../results/MYCpaper_vMOLM13/jq1&nvp2hlow.csv")